# Fitting a model to data

In [ ]:
# run first: importing our modules
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
import scipy.optimize

## 1. Fitting a line to data

Let's generate some noisy data that loosely follows a line:

In [ ]:
# create some noisy data
# first, create the time steps along the x axis: every 0.1 units for 4 units
x = np.arange(0,4,step=0.1)

# generate some noise with the same size as 'x'
noise = np.random.normal(size=np.shape(x))

# create a 'true' model with parameters a_true and b_true, somewhere between 0 and 5.
# These are the 'hidden' values we want to know.
# Note: they will be different each time you run this cell, be careful!
a_true = 5*np.random.rand()
b_true = 5*np.random.rand()

# create the y-values and add some noise.
# Question: how does the data look if you take out the noise?
y = a_true * x + b_true + noise

plt.figure(figsize=(8,5))
plt.plot(x,y,'k.',label='noisy data')
plt.legend(loc='lower right')
plt.show()

#### Assignment: Try to fit the data by hand by guessing the values of a and b until the model looks good.

In [ ]:
a=0
b=5
model = a*x + b

plt.figure(figsize=(8,5))
plt.plot(x,y,'k.',label='noisy data')
plt.plot(x,model,'-r',label='my model')
plt.legend(loc='lower right')
plt.show()

How close did you get?

In [ ]:
print("I estimated ", a, " for a, the true value was ", a_true)
print("I estimated ", b, " for b, the true value was ", b_true)

## 1a. Interactive plots!

In [ ]:
# we use a random seed to change the random result that we get each time we run the code.
# we have to put this outside the function 
np.random.seed()
rseed = np.random.randint(100)
print(rseed)

def plot_interactive_line(numdata=20, a=2., b=2., noiselevel=2.):

    # User-defined model:
    xmax=10
    x = np.linspace(-xmax, xmax, int(100*xmax))
    y = a*x + b
 
    # create random noisy data
    # get a random a between (0,5) and b between (0,5)
    np.random.seed(rseed)
    a_true = -5 + 10 * np.random.rand()
    b_true = -5 + 10 * np.random.rand()
    
    # create some synthetic GPS locations
    xdata = np.linspace(-0.9*xmax,0.9*xmax,numdata) + np.random.normal(loc=0,scale=1,size=numdata)
    
    # create the synthetic model
    ymodel = a_true*xdata + b_true
    
    #add gaussian noise: mean of 0, standard deviation of 'noiselevel'
    ynoise = np.random.normal(loc=0,scale=noiselevel,size=numdata)
    
    #get the final 'observed' data
    ydata = ymodel + ynoise
    
    # compute the model from the input parameters
    ydata_predicted = a + b*xdata
    
    # compute the total misfit ('chi-squared' statistic): 
    # this is the sum of the misfits, divided by number of data and the data noise
    chi_squared = np.sum( (ydata - ydata_predicted)**2 )/(numdata*noiselevel**2)
    
    # prepare the plot
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    
    # prepare the axes limits
    ax.set_xlim((-xmax,xmax))
    ax.set_ylim((-50,50))
    
    #ax.plot(x, y, 'bx')
    points = ax.errorbar(xdata, ydata, fmt='ko', yerr=noiselevel, label='Data',capsize=2)
    lineref = ax.plot(x, y, 'b-',label = "Model: a=%.1f, b=%.1f"%(a,b))
    ax.text(-0.9*xmax,-45,'misfit = %.2f'%chi_squared)
    
    plt.setp(lineref, linewidth=2)
    plt.legend()
    plt.grid()
    plt.show()

    return a_true, b_true, xdata, ydata


# now, we create the interactive plot with the special jupyter commands interactive() and display(). 
w = interactive(plot_interactive_line, numdata=(1,100), a=(-5.,5.), b=(-5.,5.),noiselevel=(0.5,5.))
display(w)

How close did you get this time?

In [ ]:
print("The true value of a was ", w.result[0])
print("The true value of b was ", w.result[1])

#### If the interactive code block above does not work:

Open up the anaconda prompt (search for anaconda prompt in the anaconda window), and run the following commands:

    conda install nodejs
    conda install -c conda-forge ipywidgets
    jupyter labextension install @jupyter-widgets/jupyterlab-manager


## 2. Fitting the data with python automatically

Since least-squares fitting is such a common activity, there are nice, easy functions in python to do it. Let's use one:

In [ ]:
# np.polyfit works similar to matlab polyfit(). the "order" of fitting (1 in this case) is the highest power of x.
# Fitted coefficients are returned in order of highest power to lowest.
m_fit = np.polyfit(x, y, 1)

# compute the predicted y values:
# np.polyval() does the same thing as this line:
# y_predicted = m_fit[0]*x + m_fit[1]

y_predicted = np.polyval(m_fit, x)

plt.plot(x,y,'k.',label='noisy data')
plt.plot(x,y_predicted, '-r', label='least-squares fit')
plt.legend(loc='lower right')

print('Numpy estimated values are: a =',m_fit[0],'     b =',m_fit[1])
print('The true values are:   a_true =',a_true,'b_true =',b_true)

#### More advamced fitting: non-polynomial functions

The above method is great, as long as we want to fit a linear function to our data (or a quadratic, or a cubic, etc.).

What if we want to fit something that's not a line? In this case, we need something more advanced. Here we can use `scipy.optimize.curve_fit` 

In [ ]:
# for this method, we define our linear model as a function. 
# The first argument is the x data, and the rest of the arguments are the parameters to fit.

def my_line(x,a,b):
    return a*x + b

# here, the inputs are the function name, the x data, and the y data. 
# finally p0 is the 'initial guess' for this method. 
# The outputs are m and m_cov.
# m is the model output, and mcov is the model covariance, or uncertainties.

m,mcov = scipy.optimize.curve_fit(my_line,x,y,p0=[0,0])

print(m)

# to get the predicted y values, we can just run our function with the values of m.
y_predicted=my_line(x,m[0],m[1]) 

plt.plot(x,y,'k.',label='noisy data')
plt.plot(x,y_predicted, '-r', label='least-squares fit')
plt.legend(loc='lower right')

## Bonus. Fitting a model with linear algebra

for the model function y=a*x+b, we can express this in matrix notation as follows:

    [y1] = [x1  1]  *  [a]
    [y2]   [x2  1]     [b]
    [y3]   [x3  1]
    [...]  [ ... ]  

The matrix on the left is an Nx1 column of the y-values, and on the right we have an Nx2 matrix filled with first a column of the x-values, and then a column of ones, multiplied into the 2x1 matrix with the parameters we want to fit - a and b. The result of multiplying a [Nx2] with [2x1] matrix is [Nx1], so we can see that the matrix dimensions work. 

We can actually express *any* model that is a linear function of the parameters in this way; in Geophysics this is typically written as solving the problem $Gm=d$, where $G$ is our "design matrix" (in this case, filled with ones and x-values), $d$ is our "data" matrix of the measured y-values, and $m$ is the model parameters we want to estimate; in this case, the y-intercept $a$ and slope of the line $b$.

Using the theorems of linear algebra, we can show that the solution to the matrix problem $Gm=d$ can be written as $m=(G^TG)^{-1}G^Td$. This equation is scary to look at, but in the computer we can work it out pretty quickly.


#### Step 1. First we create our 'design matrix'. For a linear model, this is a column of ones, next to a column of the x-coordinate of the data:

In [ ]:
col1 = np.ones(np.shape(x))
col2 = x
# use np.column_stack to line up arrays as columns in a matrix. There is a similar function for rows, np.row_stack.
G = np.column_stack( (col1,col2) )
print(G)

Assignment: compute the best-fitting line! Recall our equation: 

$m=(G^TG)^{-1}G^Td$

Use the numpy commands for transpose and inverse:

    np.transpose()
    np.linalg.inv()
    
And, to multiply matrices, use the .dot() method of numpy arrays, like so:

    AtimesB = A.dot(B)

Look out for typos, it's a lot to type!

In [ ]:
# compute the value of m = (G^T * G)^-1 * G^T * y
# step 1: define a new variable GT as the transpose of G


# step 2: define GTG as GT * G


# step 3: define GTGinv as GTG^-1


# step 4: GTGInvGT = GTGInv * GT


#step 5: get the final result, m, as GTGinvGT * y.


# print out the result, and compare to the true values from above.
print(m)
print(a_true,b_true)

Note, these values should be pretty close to our true values but not exactly - this is because we added noise! In general, the presence of noise means we will never be able to exactly measure the **true model** values. 

Now, let's plot the model on top of the data and see how it fits. Remember, we can get our "predicted" data (y-values of the model) with the equation $d = G * m$

In [ ]:
data_predicted = 

plt.figure(figsize=(8,5))
plt.plot(x,y,'k.',label='noisy data')
plt.plot(x,data_predicted, '-r', label='least-squares fit')
plt.legend(loc='lower right')